In [3]:
import pandas as pd
from scipy import stats
from sklearn.preprocessing import StandardScaler

In [4]:
#Step 1: Loading the Dataset
from google.colab import files


uploaded = files.upload()

Saving test.csv to test.csv


In [6]:
#Step 1: Loading the Dataset
from google.colab import files


uploaded = files.upload()

Saving train.csv to train.csv


In [7]:
data = pd.read_csv('test.csv')
data.head()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [8]:
# Step 2: Handling Missing Values
print("\nMissing values before handling:")
print(data.isnull().sum())


Missing values before handling:
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [9]:
# Fill missing 'Age' and 'Fare' values with the mean
data['Age'].fillna(data['Age'].mean(), inplace=True)
data['Fare'].fillna(data['Fare'].mean(), inplace=True)

# Fill missing 'Embarked' values with the most common value
data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)

# Fill missing 'Cabin' values with 'Unknown'
data['Cabin'].fillna('Unknown', inplace=True)

In [10]:
print("\nMissing values after handling:")
print(data.isnull().sum())



Missing values after handling:
PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64


In [11]:
#Step 3: Cleaning Irrelevant Data
# Drop columns that are not useful for prediction
data.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)


In [12]:
#Step 4: Encoding Categorical Variables
# Convert 'Sex' into numerical values
data['Sex'] = data['Sex'].map({'male': 0, 'female': 1})

# Convert 'Embarked' into numerical values using one-hot encoding
test_df = pd.get_dummies(data, columns=['Embarked'], drop_first=True)


In [13]:
#Step 5: Feature Engineering
# Create a new feature 'FamilySize' from 'SibSp' and 'Parch'
test_df['FamilySize'] = test_df['SibSp'] + test_df['Parch'] + 1

# Create a new feature 'IsAlone'
test_df['IsAlone'] = 1  # Initialize to 1
test_df['IsAlone'].loc[test_df['FamilySize'] > 1] = 0  # Update to 0 if FamilySize > 1


<ipython-input-13-2cff61d1865f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['IsAlone'].loc[test_df['FamilySize'] > 1] = 0  # Update to 0 if FamilySize > 1


In [14]:
#Step 6: Transformation
from scipy import stats

# Identify numerical features
numerical_features = ['Age', 'Fare']

# Calculate the Z-scores of the numerical features
z_scores = stats.zscore(test_df[numerical_features])

# Identify outliers as points with a Z-score > 3 or < -3
outliers = (abs(z_scores) > 3).any(axis=1)

# Remove outliers from the dataset
test_df = test_df[~outliers]


In [15]:
#Step 7: Normalization
from sklearn.preprocessing import StandardScaler

# Initialize the StandardScaler
scaler = StandardScaler()

# Normalize numerical features
numerical_features = ['Age', 'Fare', 'FamilySize']
test_df[numerical_features] = scaler.fit_transform(test_df[numerical_features])

# Display the processed test dataframe
print(test_df.head())


   PassengerId  Pclass  Sex       Age  SibSp  Parch      Fare  Embarked_Q  \
0          892       3    0  0.413606      0      0 -0.621139        True   
1          893       3    1  1.457533      1      0 -0.649915       False   
2          894       2    0  2.710246      0      0 -0.556650        True   
3          895       3    0 -0.212750      0      0 -0.592221       False   
4          896       3    1 -0.630321      1      1 -0.466422       False   

   Embarked_S  FamilySize  IsAlone  
0       False   -0.534070        1  
1        True    0.136037        0  
2       False   -0.534070        1  
3        True   -0.534070        1  
4        True    0.806143        0  


In [16]:
# Display the processed test dataframe
print(data.head())


   PassengerId  Pclass  Sex   Age  SibSp  Parch     Fare Embarked
0          892       3    0  34.5      0      0   7.8292        Q
1          893       3    1  47.0      1      0   7.0000        S
2          894       2    0  62.0      0      0   9.6875        Q
3          895       3    0  27.0      0      0   8.6625        S
4          896       3    1  22.0      1      1  12.2875        S


In [17]:

# Save the processed test dataframe to a CSV file
test_df.to_csv('processed_test.csv', index=False)